In [0]:
import os
import sys
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from google.colab import drive
root = '/gdrive'
drive.mount(root)
root = os.path.join(root, 'My Drive', '202002-j-league')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Setting Derectories and its paths

In [0]:
data_dir = os.path.join(root, 'data')
raw_dir = os.path.join(data_dir, 'raw')  # Raw data
in_dir = os.path.join(data_dir, 'input')  # Input (Processed) data
out_dir = os.path.join(data_dir, 'output')  # Output e.g. Predicted value

for d in [data_dir, raw_dir, in_dir, out_dir]:
    os.makedirs(d, exist_ok=True)

In [0]:
import requests

for competition_year in range(2009, 2020 + 1):
    csv_path = os.path.join(raw_dir, f'{competition_year}.csv')
    if os.path.isfile(csv_path):
        continue

    url = f'https://data.j-league.or.jp/SFMS01/search?competition_years={competition_year}'
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    bs = BeautifulSoup(response.text, 'html.parser')

    table = bs.select('table')[0]

    def to_csv_line(raw_line: str) -> str:
        values = raw_line.findAll('td')  # data
        if not values:
            values = raw_line.findAll('th') # column
        values = list(map(lambda x: x.get_text(), values))
        trans = str.maketrans({s: '' for s in [' ', ',', '\r', '\n', '\t']})
        values = map(lambda x: x.translate(trans), values)
        return '{}\n'.format(','.join(values))

    lines = table.findAll('tr')
    lines = list(map(lambda x: to_csv_line(x), lines))

    csv_path = os.path.join(raw_dir, f'{competition_year}.csv')
    with open(csv_path, 'w') as f:
        for line in lines:
            f.write(line)
    #time.sleep(60)